In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\Base'
MODEL_PATH = 'C:\Github\DL_Study\RNN'

sys.path.insert(0, MODULE_PATH)
sys.path.insert(0, MODEL_PATH)

import numpy
from config import *
from seq2seq_np import *

In [2]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = PeekySeq2Seq(time_size=24, hidden_size=64, feature_size=18)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [3]:
# dataset
import zipfile, requests, io
import pandas as pd
import numpy
import time
from datetime import datetime

numpy.random.seed(42)
np.random.seed(42)

df_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H.%M.%S')
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00274/NEW-DATA.zip'
r = requests.get(data_url)
files = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.concat(
    [pd.read_csv(files.open(i), sep=' ') for i in files.namelist()]
)
cols = df.columns.tolist()
cols = cols[2:]+cols[:2]
df.columns = cols
df.drop(df.columns[[0, 1, 18, 19, 20, 24, 25]], axis = 1, inplace=True)
df.drop(columns=df.columns[[1]], axis=1, inplace=True)      # for target column 3
df.head()

,3:Temperature_Comedor_Sensor,5:Weather_Temperature,6:CO2_Comedor_Sensor,7:CO2_Habitacion_Sensor,8:Humedad_Comedor_Sensor,9:Humedad_Habitacion_Sensor,10:Lighting_Comedor_Sensor,11:Lighting_Habitacion_Sensor,12:Precipitacion,13:Meteo_Exterior_Crepusculo,14:Meteo_Exterior_Viento,15:Meteo_Exterior_Sol_Oest,16:Meteo_Exterior_Sol_Est,17:Meteo_Exterior_Sol_Sud,18:Meteo_Exterior_Piranometro,22:Temperature_Exterior_Sensor,23:Humedad_Exterior_Sensor,24:Day_Of_Week
0,18.1875,0.0,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,1.42625,9690.24,12604.20,95436.8,758.880,18.1150,48.3750,2.0
1,18.4633,6.8,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.59200,11022.00,10787.20,95436.8,762.069,18.4147,47.8080,2.0
2,18.7673,17.0,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,1.89133,13960.50,9669.63,95398.6,766.251,18.8533,47.4320,2.0
3,19.0727,18.0,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,1.82800,18511.20,9648.13,95360.3,766.037,19.2907,47.0240,2.0
4,19.3721,20.0,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,2.36071,26349.00,9208.32,95354.9,762.743,19.7400,45.4743,2.0


In [4]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 18
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var10(t-24),...,var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var18(t-1),var1(t)
24,18.1875,0.0,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,...,615.957,1.088670,3024.90,2229.250,2042.880,24.66530,19.9760,45.8293,2.0,21.7133
25,18.4633,6.8,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,...,615.360,0.844667,2207.74,1576.960,1512.450,15.57000,19.7560,47.0693,2.0,21.6600
26,18.7673,17.0,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,...,614.592,1.442000,1017.86,222.208,293.888,5.00933,19.5273,49.5467,2.0,21.5960
27,19.0727,18.0,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,...,585.984,0.969333,0.00,0.000,0.000,-2.93933,19.3013,52.2027,2.0,21.5307
28,19.3721,20.0,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,...,150.851,0.798667,0.00,0.000,0.000,-3.90400,19.0413,53.8053,2.0,21.4493


In [5]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)
    
    # check start time
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check duration
    duration = time.time()-start_time
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}, times : {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==   error(RMSE):3.428110548839318
 == train [30, 64, 0.001] model ==   error(RMSE):3.9822459547759106
train-size:536, val-size:133, test-size:342
best_model => error(rmse) : 3.35, param:[30, 64, 0.01], times : 12.642

fold : 2/10
 == train [30, 64, 0.01] model ==   error(RMSE):3.5036890477210054
 == train [30, 64, 0.001] model ==   error(RMSE):3.5050729251760018
train-size:809, val-size:202, test-size:342
best_model => error(rmse) : 3.05, param:[30, 64, 0.01], times : 19.791

fold : 3/10
 == train [30, 64, 0.01] model ==   error(RMSE):3.3310623808206605
 == train [30, 64, 0.001] model ==   error(RMSE):3.352001684461265
train-size:1083, val-size:270, test-size:342
best_model => error(rmse) : 3.84, param:[30, 64, 0.01], times : 26.847

fold : 4/10
 == train [30, 64, 0.01] model ==   error(RMSE):3.8260308284622466
 == train [30, 64, 0.001] model ==   error(RMSE):3.8776826237796396
train-size:1356, val-s

In [6]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=9.001162210217704, std=2.6646077802228225
RMSE: mean=2.967635320398351, std=0.44079793028311715
MAE: mean=2.408401079781599, std=0.36233990578971975

[training time]
mean : 42.458846783638, last:71.90817403793335
